In [1]:
import numpy as np
import pandas as pd
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
import matplotlib.pyplot as plt
%matplotlib inline

# 5. Feature Engineering
- 주가에 대한 예측을 위해서 필요한 Feature Enginnering을 한다.
- pickle 형식의 데이터를 불러오고 전처리를 진행한다.

## 5-1) 데이터 불러오기

In [31]:
df_stocks = pd.read_pickle('./data/pickled_ten_year_filtered_lead_para.pkl')

## 5-2) 전처리
- 날짜 데이터 년-월-일 형태로 변경
- 뉴스 제목 데이터를 병합하면서 생긴 주가 결측치 대체
- 수정 종가를 정수 형태로 변환
- 뉴스 헤드라인에서 가장 왼쪽에 놓인 점을 제거

In [32]:
df_stocks.isnull().sum()

close        98
adj close    98
articles      0
dtype: int64

In [33]:
df_stocks = df_stocks.interpolate()
df_stocks.isnull().sum()

close        0
adj close    0
articles     0
dtype: int64

In [34]:
# 수정 종가를 정수형태로 변환
df_stocks['prices'] = df_stocks['adj close'].apply(np.int64)

In [35]:
# prices와 articles만 선택
df_stocks = df_stocks[['prices', 'articles']]
df_stocks.head()

,prices,articles
2010-01-01 00:00:00,10467,. Time Warner and Fox Reach Deal for Cable Dis...
2010-01-02 00:00:00,10506,. Even Better than Pepperoni Pizza.... Transit...
2010-01-03 00:00:00,10544,. Media Equation: And a Tablet Will Lead Them ...
2010-01-04 00:00:00,10583,". In Galleon Cases, Timing Could Be Everything..."
2010-01-05 00:00:00,10572,. Rajaratnam Expected to Face Additional Charg...


In [37]:
df_stocks.index = pd.DatetimeIndex(df_stocks.index,)

In [38]:
df_stocks.index

DatetimeIndex(['2010-01-01', '2010-01-02', '2010-01-03', '2010-01-04',
               '2010-01-05', '2010-01-06', '2010-01-07', '2010-01-08',
               '2010-01-09', '2010-01-10',
               ...
               '2018-10-28', '2018-11-18', '2018-11-22', '2018-11-24',
               '2018-11-30', '2018-12-02', '2018-12-24', '2018-12-26',
               '2018-12-28', '2018-12-30'],
              dtype='datetime64[ns]', length=3750, freq=None)

In [39]:
# 뉴스 헤드라인에서 가장 왼쪽에 있는 점을 제거
df_stocks['articles'] = df_stocks['articles'].map(lambda x: x.lstrip('.-'))
df_stocks.head()

,prices,articles
2010-01-01,10467,Time Warner and Fox Reach Deal for Cable Dist...
2010-01-02,10506,Even Better than Pepperoni Pizza.... Transit ...
2010-01-03,10544,Media Equation: And a Tablet Will Lead Them ....
2010-01-04,10583,"In Galleon Cases, Timing Could Be Everything...."
2010-01-05,10572,Rajaratnam Expected to Face Additional Charge...


## 5-3) Feature 선택
- Feature 선택은 머신러닝에서 가장 중요한 측면 중 하나이다.
- 종가가 아닌 수정종가를 선택한 이유는 수정 종가가 주식, 뮤추얼 펀드, 배당금 등 DJIA지수에 대한 거래일의 최종가격이 무엇인지에 대해 더 잘 보여주기 때문이다.

In [40]:
df = df_stocks[['prices']].copy()
df.head()

,prices
2010-01-01,10467
2010-01-02,10506
2010-01-03,10544
2010-01-04,10583
2010-01-05,10572


## 5-4) 정서 분석
- 정서 분석(sentiment analysis)을 구현하기 위해 nltk에 내장된 정서 분석 모듈을 사용한다.
- 부정 정서, 긍정 정서, 복합 정서 점수를 얻는다.
- Lexicon 기반 접근법을 사용하여 각 문자의 단어가 분석되고 sentiwordnet 점수를 기반으로 각 단어에 특정 정서 점수가 부여한다.
- 마지막으로 집계된 문장 수준 점수가 결정된다.

In [41]:
# 데이터 프레임에 새로운 컬럼 추가
df["compound"] = ''
df["neg"] = ''
df["neu"] = ''
df["pos"] = ''

In [42]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/pirl/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [43]:
import unicodedata

sid = SentimentIntensityAnalyzer()

In [46]:
df_stocks.T.index

Index(['prices', 'articles'], dtype='object')

In [ ]:
for date, row in df_stocks.T.iteritems():
    try:
        sentence = unicodedata.normalize('NFKD', df_stocks.loc[date, 'articles'])
        ss = sid.polarity_scores(sentence)
        df.set_value(date, 'compound', ss['compound'])
        df.set_value(date, 'neg', ss['neg'])
        df.set_value(date, 'neu', ss['neu'])
        df.set_value(date, 'pos', ss['pos'])
    except TypeError:
        print(df_stocks.loc[date, 'articles'])
        print(date)

In [49]:
# 뉴욕 타임즈 뉴스 기사 데이터셋에 대한 정서 점수을 얻었다.
df.head()

,prices,compound,neg,neu,pos
2010-01-01,10467,-0.9853,0.147,0.748,0.104
2010-01-02,10506,-0.8377,0.122,0.778,0.1
2010-01-03,10544,-0.9858,0.184,0.713,0.103
2010-01-04,10583,-0.9941,0.128,0.779,0.093
2010-01-05,10572,-0.9962,0.123,0.789,0.088


# 6. 머신러닝 알고리즘 선택
- 주가 예측은 시계열 분석에 대한 문제이다.
- Baseline Model은 랜덤포레스트 회귀로 정하고 진행한다.

## 6-1) Train set과 Test set 분할

In [50]:
# 2010년부터 2017년까지 8년분 데이터를 Train set으로 지정한다.
train_start_date = '2010-01-01'
train_end_date = '2017-12-31'
# 2018년부터 2019년까지 2년분 데이터를 Test set으로 지정한다.
test_start_date = '2018-01-01'
test_end_date = '2019-12-31'
train = df.loc[train_start_date : train_end_date]
test = df.loc[test_start_date:test_end_date]

## 6-2) Train, Test set 예측 레이블 분할
- Train 및 Test set을 분리할 때 수정 종가를 별도로 저장해야한다.
- 이 가격 값은 Train data의 레이블이며, 이 모델은 실제 가격을 레이블 형식으로 제공하므로 지도 학습이된다.

In [51]:
y_train = pd.DataFrame(train['prices'])
y_test = pd.DataFrame(test['prices'])

## 6-3) 정서 점수를 Numpy 배열로 변환
- 학습을 시작하기 전에 정서 분석 점수들을 넘파이 배열 형식으로 변환한다.
- 가격 특성을 예측 레이블로 설정하면 특징 벡터에 정서 점수와 날짜만 포함되기 때문이다.

In [ ]:
sentiment_score_list = []
for date, row in train.T.iteritems():
    #sentiment_score = np.asarray([df.loc[date, 'compound'],
                                  #df.loc[date, 'neg'],df.loc[date, 'neu'],df.loc[date, 'pos']])
    sentiment_score = np.asarray([df.loc[date, 'neg'],df.loc[date, 'pos']])
    sentiment_score_list.append(sentiment_score)
numpy_df_train = np.array(sentiment_score_list)

In [86]:
sentiment_score_list = []
for date, row in test.T.iteritems():
    sentiment_score = [df.loc[date, 'neg'],df.loc[date, 'pos']]
    sentiment_score_list.append(sentiment_score)
numpy_df_test = np.asarray(sentiment_score_list)

In [ ]:
from treeinterpreter import treeinterpreter as ti
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report,confusion_matrix

rf = RandomForestRegressor()
rf.fit(numpy_df_train, y_train)

## 6-4) 머신러닝 모델 훈련